In [1]:
# read saved results (tables) of old runs
# combine multiple runs
# make significance plots

import sys
import pandas as pd
import numpy as np
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
import pandas as pd
import core_tools as ct
import core_calib as cal
from scipy import stats

In [2]:
main_path = './Data manipulation/results'
# exp_names = ["1692118038_T10", "1692121657_T50", "1692130088_T100", "1692161204_O50", "1692172225_O100"] # , "1692107939_Z8T0"
# exp_names = ["1692351821_RFseed"]
exp_names = ["1697535775_bostrom_laplace0", "1697532962_bostrom_laplace1"]

In [3]:
# to combine Calib and OOB runs in one CD plot
tables_all = {}
for exp in exp_names:
    path = f"{main_path}/{exp}"
    dict_data, tables, params = ct.read_results(path)
    if len(tables_all) == 0:
        tables_all = tables
    else: # combine tables
        for table_key in tables.keys():
            col_names = np.array((tables[table_key].columns))
            col_names += "_L"
            tables[table_key].columns = col_names
            # print(tables[table_key])
            # print(f"key {table_key} col_names", col_names)
            tables_all[table_key] = pd.concat([tables_all[table_key], tables[table_key]], axis=1)
            # print(tables_all[table_key])

# tables_all = ct.add_rank_mean(tables_all)
tables_all["brier"]

# ct.res_statistics(tables_all, params["metrics"], ".")

,RF_d,RF_opt,Platt,ISO,Beta,CRF,VA,tlr,Rank,RF_large,RF_d_L,RF_opt_L,Platt_L,ISO_L,Beta_L,CRF_L,VA_L,tlr_L,Rank_L,RF_large_L
Data,,,,,,,,,,,,,,,,,,,,
vertebral,0.117213,0.109767,0.123441,0.128252,1.243805e-01,0.122898,0.123197,0.127342,0.145892,0.113132,0.114781,0.109567,0.122329,1.279889e-01,1.226386e-01,0.121783,0.121483,0.127342,0.145892,0.113263
wilt,0.014438,0.012460,0.013801,0.013764,1.478391e-02,0.013067,0.013370,0.014327,0.017308,0.011878,0.014712,0.012603,0.013664,1.364958e-02,1.327476e-02,0.013141,0.013339,0.014327,0.017308,0.012098
parkinsons,0.081713,0.095847,0.102867,0.108309,1.069762e-01,0.102712,0.101672,0.108585,0.129498,0.091117,0.084854,0.098185,0.103012,1.082900e-01,1.071657e-01,0.103389,0.102437,0.108585,0.129498,0.094151
heart,0.145747,0.139291,0.141138,0.149394,1.426988e-01,0.143758,0.143023,0.143984,0.153287,0.131516,0.139599,0.140068,0.140052,1.501324e-01,1.414483e-01,0.141488,0.144484,0.143984,0.153287,0.133759
wdbc,0.036576,0.035012,0.038915,0.040399,3.976248e-02,0.039213,0.039789,0.039346,0.055923,0.032778,0.036636,0.036230,0.038650,4.229922e-02,3.905583e-02,0.039175,0.040838,0.039346,0.055923,0.034460
bank,0.007246,0.007627,0.007689,0.009088,8.699909e-03,0.007983,0.008760,0.009767,0.024540,0.006213,0.008996,0.009011,0.008596,1.040107e-02,9.566256e-03,0.009324,0.009802,0.009767,0.024540,0.007889
ionosphere,0.061537,0.063178,0.066936,0.074845,6.854745e-02,0.065587,0.070847,0.068498,0.087248,0.059081,0.063372,0.065642,0.066350,7.369079e-02,6.789999e-02,0.066800,0.070030,0.068498,0.087248,0.062588
HRCompetencyScores,0.071000,0.070280,0.075214,0.079092,8.000135e-02,0.074513,0.076410,0.077433,0.088433,0.068921,0.069023,0.070447,0.074129,7.824286e-02,7.674849e-02,0.072692,0.075688,0.077433,0.088433,0.069487
spambase,0.045702,0.045860,0.044899,0.045532,5.049232e-02,0.046079,0.045336,0.049635,0.046119,0.041493,0.047607,0.048555,0.044896,4.549387e-02,4.498241e-02,0.046657,0.045244,0.049635,0.046119,0.045310


In [4]:
test_table = tables_all["brier"][["RF_large", "RF_large_L"]]
# test_table = tables_all["acc"][["RF_opt", "RF_opt_L"]]
test_table

,RF_large,RF_large_L
Data,,
vertebral,0.113132,0.113263
wilt,0.011878,0.012098
parkinsons,0.091117,0.094151
heart,0.131516,0.133759
wdbc,0.032778,0.034460
bank,0.006213,0.007889
ionosphere,0.059081,0.062588
HRCompetencyScores,0.068921,0.069487
spambase,0.041493,0.045310


In [5]:


# Create two sample arrays
L0 = np.array(test_table.iloc[:, 0])
L1 = np.array(test_table.iloc[:, 1])

result = L0 - L1 # for loss comparison
# result = L1 - L0 # for acc comparison
result[result > 0] = 1
result[result < 0] = 0
n_trials = len(result) 
observed_successes = int(sum(result))

print("n_trials", n_trials)
print("observed_successes", observed_successes)

result = stats.binomtest(observed_successes, n=n_trials, p=0.5, alternative='two-sided')
p_value = result.pvalue

print("p_value", p_value)

alpha = 0.05

if p_value < alpha:
    print("Reject the null hypothesis. The observed frequency is significantly different from the expected frequency.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference.")



n_trials 20
observed_successes 1
p_value 4.00543212890625e-05
Reject the null hypothesis. The observed frequency is significantly different from the expected frequency.


In [6]:

# # Perform an independent samples t-test
# t_stat, p_value = stats.ttest_ind(group1, group2)

# # Print the results
# print("T-Statistic:", t_stat)
# print("P-Value:", p_value)

# # Determine if the difference is statistically significant (using a significance level, e.g., 0.05)
# if p_value < 0.05:
#     print("There is a significant difference between the two groups.")
# else:
#     print("There is no significant difference between the two groups.")